In [2]:
import nltk,os
import requests
from libs import Stanford
from libs import TweetRepository
import pyprind
nltk.__version__

'3.1'

Se invoca a la API Rest para el control del servidor NER.

In [3]:
baseUrl = 'http://ner-server:5000'

In [4]:
url = baseUrl + '/ner/models'
response = requests.get(url)
print 'Modelos NER disponibles',response.content

Modelos NER disponibles {
  "models": [
    "english.muc.7class.distsim.crf.ser.gz", 
    "example.serialized.ncc.ncc.ser.gz", 
    "english.conll.4class.distsim.crf.ser.gz", 
    "spanish.ancora.distsim.s512.crf.ser.gz", 
    "custom-ner-model.ser.gz", 
    "english.all.3class.distsim.crf.ser.gz"
  ]
}



In [5]:
url = baseUrl + '/ner/start/spanish.ancora.distsim.s512.crf.ser.gz'
response = requests.get(url)
print 'Iniciar',response.content

Iniciar {
  "message": "Esta iniciado"
}



In [6]:
url = baseUrl + '/ner/status'
response = requests.get(url)
print 'Status NER',response.content

Status NER {
  "status": "ON"
}



In [7]:
url = baseUrl + '/pos/start'
response = requests.get(url)
print 'Iniciar POS',response.content

Iniciar POS {
  "message": "Esta iniciado"
}



In [8]:
url = baseUrl + '/pos/status'
response = requests.get(url)
print 'Status POS',response.content

Status POS {
  "status": "ON"
}



Se invoca el cliente para la comunicación con el servidor NER a travez de Sockets.
Se invoca el cliente para la comunicación con la colección de tweets-

In [9]:
stanford_client = Stanford.Stanford(baseUrl.split('//')[1].split(':')[0])
dbdb = os.getenv('MONGO_DATABASE',None)
dbhost = os.getenv('MONGO_HOST',None)
dbport = os.getenv('MONGO_PORT',None)
dbuser = os.getenv('MONGO_USER',None)
dbpass = os.getenv('MONGO_PASS',None)
dbcoll = "krypton-imported-data"#os.getenv('MONGO_COLL',None)
tws = TweetRepository.TweetRepository(dbcoll,dbhost,dbuser,dbpass,dbport,dbdb)

In [10]:
import json
import urllib
import urllib2

def searchStreetsFromName(nombre_via):
    calles = []
    r = urllib2.urlopen('http://www.montevideo.gub.uy/ubicacionesRest/calles/?nombre='+nombre_via)
    web_pg = r.read()
    j = json.loads(web_pg)
    for vias in j:
        calles.append(vias)
    return calles
def searchIntersection(street_code_1,street_code_2):
    try:
        r = urllib2.urlopen('http://www.montevideo.gub.uy/ubicacionesRest/esquina/'+str(street_code_1)+'/'+str(street_code_2))
        web_pg = r.read()
        j = json.loads(web_pg)
        return j
    except urllib2.HTTPError, e:
        return None

In [11]:
def searchPossibleLocations(data):
    sets_of_streets = {}
    for noun in stanford_client.nouns(data['text']):
        streets = []
        for street in searchStreetsFromName(noun):
            streets.append(street)
        sets_of_streets[noun] = streets
    #Damn
    possible_locations = []
    alredy_searched = {}
    for noun_i in sets_of_streets.keys():
        go_with_these_nouns = True
        for noun_j in sets_of_streets.keys():
            streets_i = sets_of_streets[noun_i]
            streets_j = sets_of_streets[noun_j]
            for street_i in streets_i:
                for street_j in streets_j:
                    str_code_i = street_i['codigo']
                    str_code_j = street_j['codigo']
                    if str_code_i != str_code_j:
                        if not alredy_searched.get(str(str_code_i)+str(str_code_j),None):
                            if not alredy_searched.get(str(str_code_j)+str(str_code_i),None):
                                intersection = searchIntersection(str_code_i,str_code_j)
                                alredy_searched[str(str_code_i)+str(str_code_j)] = True
                                alredy_searched[str(str_code_j)+str(str_code_i)] = True
                                if intersection:
                                    possible_locations.append((street_i,street_j,intersection))
                                    go_with_these_nouns = False
                                    break
                                    #print intersection
                if not go_with_these_nouns:
                    break
            if not go_with_these_nouns:
                    break
    return possible_locations

In [12]:
from pyproj import Proj, transform
def transformCoordinates(x,y):
    #Sistema de la IM
    inProj = Proj(init='epsg:32721')
    outProj = Proj(init='epsg:4326')
    x2,y2 = transform(inProj,outProj,x,y)
    return (x2,y2)

Usando el POS Tagger y el NER tagger, se le asigna a cada palabra una clase candidata. Se asigna LABEL cuando es un nombre propio que no se puede clasificar.

In [14]:
#A partir de data de la BD, (deberia tener una bandera geo-tagged = false)
todos = tws.findData({"krypton_category":"MUY_UTIL"})
bar = pyprind.ProgBar(len(todos))
palabras_anotadas = []
for data in todos[1:4]:
    print data['text']
    #Uso ner para quedarme con los nouns ?
    for loc in searchPossibleLocations(data):
        print loc[0][u'nombre'],loc[1][u'nombre']
        print transformCoordinates(loc[2][u'geoJSON'][u'coordinates'][0],loc[2][u'geoJSON'][u'coordinates'][1])
    bar.update()

0%  100%
[   ]

@quejasya @montevideoIM @Dmartinez_uy Acevedo Diaz y Palmar #limpieza #basura https://t.co/pJWjZ3dP8P
ACEVEDO DIAZ PSJE DE LA VIA (ACEVEDO DIAZ)
(-56.16771485186318, -34.89153315524597)
21 DE SETIEMBRE ACEVEDO DIAZ
(-56.165581829650996, -34.911274652843)
PALMAR ACEVEDO DIAZ
(-56.16668616389682, -34.90107706324848)
ABIPONES ANDRES LERENA ACEVEDO
(-56.113422351932044, -34.83204266065045)
ACEVEDO DIAZ ANA MONTERROSO DE LAVALLEJA
(-56.166803061545814, -34.90001762545476)
Urquiza y Berro @quejasya 
Contenedores hermeticos ya!!! https://t.co/06XnvkNIzO
ADOLFO BERRO

[## ] | ETA: 00:04:44

 PSJE PNAL (ADOLFO BERRO Y LUCAS OBES)
(-56.207395074976, -34.864005489676906)
GRAL URQUIZA PTE BERRO
(-56.16324780747015, -34.89139171117354)


In [ ]:
from pyproj import Proj, transform
inProj = Proj(init='epsg:32721')
outProj = Proj(init='epsg:4326')
x1,y1 = 579813.2135, 6142453.9517
x2,y2 = transform(inProj,outProj,x1,y1)
print x2,y2

##Cargar datos de calles en mongo local

In [37]:
from pymongo import MongoClient
from pymongo import ASCENDING
dbdb = 'krypton'
dbhost = 'ner-mongo'
dbuser = 'krypton'
dbpass = 'krypton_pass'
otherClient = MongoClient('mongodb://'+dbhost+'/'+dbdb)
krypton_db = otherClient[dbdb]
v_mdg_vias = krypton_db['v-mdg-vias']
sig_cruces_vias = krypton_db['sig-cruces-vias']

In [38]:
v_mdg_vias.drop_indexes()
v_mdg_vias.create_index([('properties.NOM_CALLE', 'text')], default_language='spanish')
sig_cruces_vias.drop_indexes()
sig_cruces_vias.create_index([('properties.COD_NOMBRE',ASCENDING),('properties.COD_NOM_01',ASCENDING)])

u'properties.COD_NOMBRE_1_properties.COD_NOM_01_1'

In [43]:
def findStreets(text):
    return list(v_mdg_vias.find( { '$text': { '$search': text } }))

In [45]:
def findIntersection(street_code_1, street_code_2):
    return list(sig_cruces_vias.find( { 'properties.COD_NOMBRE': street_code_1,  'properties.COD_NOM_01': street_code_2 }))

In [30]:
for c in todos:
    print c[u'properties'][u'NOM_CALLE'],c[u'properties'][u'COD_NOMBRE']

DR JUSTO MONTES PAREJA 4553.0
DR JUSTO MONTES PAREJA 4553.0
DR JUSTO MONTES PAREJA 4553.0
DR JUSTO MONTES PAREJA 4553.0
DR JUSTO MONTES PAREJA 4553.0
DR JUSTO MONTES PAREJA 4553.0
DR JUSTO MONTES PAREJA 4553.0
DR JUSTO MONTES PAREJA 4553.0
DR JUSTO MONTES PAREJA 4553.0
DR JUSTO MONTES PAREJA 4553.0
CNO MONTE SOSA 4555.0
CNO REY DEL MONTE 5956.0
CNO REY DEL MONTE 5956.0
CNO REY DEL MONTE 5956.0
CNO REY DEL MONTE 5956.0
CNO MONTE SOSA 4555.0
CNO REY DEL MONTE 5956.0
CNO REY DEL MONTE 5956.0
CNO REY DEL MONTE 5956.0
CNO REY DEL MONTE 5956.0
CNO MONTE SOSA 4555.0
CNO MONTE SOSA 4555.0
CNO MONTE SOSA 4555.0
CNO REY DEL MONTE 5956.0
CNO REY DEL MONTE 5956.0
CNO MONTE SOSA 4555.0
CNO REY DEL MONTE 5956.0
CNO REY DEL MONTE 5956.0
CNO MONTE SOSA 4555.0
CNO REY DEL MONTE 5956.0
GATO MONTES 1385.0
ENRIQUE MONTES 4554.0
MONTE CASEROS 4536.0
MONTE CASEROS 4536.0
MONTE CASEROS 4536.0
CNO REY DEL MONTE 5956.0
MONTE CASEROS 4536.0
CNO MONTE SOSA 4555.0
GATO MONTES 1385.0
GATO MONTES 1385.0
MONTE CASER

In [47]:
todos = findIntersection(3298,5956)

In [48]:
for c in todos:
    print c

{u'geometry': {u'type': u'Point', u'coordinates': [-56.27884664502937, -34.809330813703575]}, u'_id': ObjectId('57f587049d6c5bd84902d24e'), u'type': u'Feature', u'properties': {u'UCREA': u'usersig', u'COD_DEPTO': 1, u'NOM_CAL_01': u'CNO REY DEL MONTE', u'GID': 37779119.0, u'IDAUDITORI': None, u'FCREA': u'2016-09-19 03:15:01.235645', u'COD_NOM_01': 5956.0, u'UACT': u'usersig', u'COD_NOMBRE': 3298.0, u'FACT': u'2016-09-19 03:15:01.235645', u'NOM_CALLE_': u'CNO HAMILTON'}}
{u'geometry': {u'type': u'Point', u'coordinates': [-56.27884664502937, -34.809330813703575]}, u'_id': ObjectId('57f58c868c14a791aa05cd29'), u'type': u'Feature', u'properties': {u'UCREA': u'usersig', u'COD_DEPTO': 1, u'NOM_CAL_01': u'CNO REY DEL MONTE', u'GID': 37779119.0, u'IDAUDITORI': None, u'FCREA': u'2016-09-19 03:15:01.235645', u'COD_NOM_01': 5956.0, u'UACT': u'usersig', u'COD_NOMBRE': 3298.0, u'FACT': u'2016-09-19 03:15:01.235645', u'NOM_CALLE_': u'CNO HAMILTON'}}
{u'geometry': {u'type': u'Point', u'coordinates': 